In [1]:
!test -f test.grib || wget https://github.com/ecmwf/emohawk/raw/main/docs/examples/test.grib

In [2]:
import emohawk

fs = emohawk.load_from("file", "test.grib")

In [3]:
type(fs)

emohawk.sources.file.File

In [4]:
len(fs)

2

In [5]:
fs.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,2t,surface,0,20200513,1200,0,an,0,regular_ll
1,ecmf,msl,surface,0,20200513,1200,0,an,0,regular_ll


#### Slicing

In [6]:
fs[1]

GribField(msl,None,20200513,1200,0,0)

In [7]:
g = fs[1:2]
type(g)

emohawk.readers.grib.index.MaskFieldSet

In [8]:
g.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,msl,surface,0,20200513,1200,0,an,0,regular_ll


In [9]:
g = fs[-1]
g

GribField(msl,None,20200513,1200,0,0)

#### Values

values always flattens the results. to_numpy() also does so by default but we can get a 2d shape (e.g. for lat-lon grids) using flat_field=False.  

In [10]:
v = fs[0].values
v.shape

(209,)

In [11]:
v = fs[0].to_numpy()
v.shape

(209,)

In [12]:
v = fs[0].to_numpy(flat_field=False)
v.shape

(11, 19)

In [13]:
v = fs.values
v.shape

(2, 209)

In [14]:
v = fs.to_numpy()
v.shape

(2, 209)

#### Metadata

coords cannot ba called on a File:

In [15]:
try:
    fs.coords
except Exception as e:
    print(e)

'File' object has no attribute 'coords'


In [16]:
fs[0].field_metadata()

{'north': 73.0,
 'south': 33.0,
 'west': -27.0,
 'east': 45.0,
 'south_north_increment': 4.0,
 'west_east_increment': 4.0,
 'shortName': '2t',
 'units': 'K',
 'paramId': '167',
 'shape': (11, 19)}

In [17]:
fs[0].metadata("typeOfLevel")

'surface'

In [18]:
fs.metadata("level")

[0, 0]

In [19]:
fs[0].metadata(["level", "paramId"])

[0, 167]

In [20]:
fs.metadata(["level", "paramId"])

[[0, 167], [0, 151]]

Key qualifiers work.

In [21]:
fs[0].metadata(["centre", "centre:l", "centre:int", "centre:s", "centre:str"])

['ecmf', 98, 98, 'ecmf', 'ecmf']

Metadata access with [] only works on individual fields:

In [22]:
fs[0]["centre"]

'ecmf'

In [23]:
try:
    v = fs["centre"]
except Exception as e:
    print(e)

only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices


#### Xarray

In [24]:
ds = fs.to_xarray()
ds

<xarray.Dataset>
Dimensions:     (number: 1, time: 1, step: 1, surface: 1, latitude: 11,
                 longitude: 19)
Coordinates:
  * number      (number) int64 0
  * time        (time) datetime64[ns] 2020-05-13T12:00:00
  * step        (step) timedelta64[ns] 00:00:00
  * surface     (surface) float64 0.0
  * latitude    (latitude) float64 73.0 69.0 65.0 61.0 ... 45.0 41.0 37.0 33.0
  * longitude   (longitude) float64 -27.0 -23.0 -19.0 -15.0 ... 37.0 41.0 45.0
    valid_time  (time, step) datetime64[ns] ...
Data variables:
    t2m         (number, time, step, surface, latitude, longitude) float32 ...
    msl         (number, time, step, surface, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2023-01-31T17:32 GRIB to CDM+CF via cfgrib-0.9.1...